### Step 37. 텐서를 다루다.

지금까지의 변수는 주로 '스칼라'를 다루었으나, 머신러닝 데이터로는 벡터나 행렬 등이 '텐서'가 주로 쓰인다. 이번 단계에서는 텐서를 사용할 때의 주의점을 알아보면서 Dezero를 확장한다.

Variable의 data에는 ndarray 타입의 넘파이 객체가 대입되는데, 넘파이에는 브로드캐스팅 기능이 있어, 원소별 계산을 하는 함수는 입력값이 스칼라든, 행렬이든 기존의 Dezero도 계산과 역전파를 진행할 수 있다. 요약하면 다음과 같다.

* 원소별 연산을 수행하는 함수(add, sin 등)는 입출력 데이터가 스칼라라고 가정하고, 순전파와 역전파를 구현할 수 있다.
* 이 경우 텐서를 입력해도 역전파가 올바르게 성립한다.

In [ ]:
import numpy as np
import dezero.functions as F
from dezero import Variable

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sin(x)
print(y)

y.backward()
print(x.grad)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])
variable([[ 0.54030231 -0.41614684 -0.9899925 ]
          [-0.65364362  0.28366219  0.96017029]])


### Step 38. 형상 변환 함수

앞으로는 원소별로 계산하지 않는 함수에 대해 살펴보겠다. 이번 단계에서는 그 첫걸음으로 두 가지 함수를 구현한다. 텐서의 형상을 변환하는 reshape 함수와, 행렬을 전치하는 transpose 함수이다.

In [ ]:
from dezero import Function

class Reshape(Function):
  def __init__(self, shape):
    self.shape = shape

  def forward(self, x):
    self.x_shape = x.shape
    y = x.reshape(self.shape)
    return y

  def backward(self, gy):
    return reshape(gy, self.x_shape)

def reshape(x, shape):
  if x.shape == shape:
    return as_variable(x)
  return Reshape(shape)(x)

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = reshape(x, (6,))
print(y)
y.backward(retain_grad=True)
print(x.grad)

variable([1 2 3 4 5 6])
variable([[1 1 1]
          [1 1 1]])


이어서, Dezero의 Variable 변수로도 넘파이에서처럼 reshape을 활용할 수 있도록 Variable 클래스에 코드를 추가해준다.

```python
class Variable:
  ...
  def reshape(self, *shape):
    if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
      shape = shape[0]
    return dezero.functions.reshape(self, shape)
```

In [ ]:
# test
x = Variable(np.random.randn(1, 2, 3))
y = x.reshape(2, 3)
print(y)

variable([[ 1.66995045  0.05488989  0.80820596]
          [ 0.66173862 -1.59093227 -0.88178582]])


다음은 행렬의 전치, transpose 함수를 구현해 보도록 하겠다.

In [ ]:
from dezero import Function

class Transpose(Function):
  def forward(self, x):
    y = x.T
    return y

  def backward(self, gy):
    gx = transpose(gy)
    return gx

def transpose(x):
  return Transpose()(x)

# test
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = transpose(x)
print(y)
y.backward()
print(x.grad)

variable([[1 4]
          [2 5]
          [3 6]])
variable([[1 1 1]
          [1 1 1]])


Variable 함수에서, 이 transpose 함수를 원활히 사용할 수 있도록 다음 함수를 추가한다.

```python
class Variable:
  ...
  @property
  def T(self):
    return dezero.functions.transpose(self)
```

In [2]:
# test
x = Variable(np.random.randn(2, 3))
y = x.T
print(y)

variable([[-0.20722328  0.59093764]
          [-0.04202923 -0.3450468 ]
          [-1.31850134  1.27172508]])


In [1]:
import os, sys
from google.colab import drive

drive.mount('/content/drive')
%cd drive/MyDrive/Colab\ Notebooks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


### Step 39. 합계 함수

이번에는 numpy의 sum 함수를 구현해 보도록 하겠다. 보통 numpy에서 sum 함수는 다음과 같이 사용된다.

```python
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]])

y = x - x.sum(axis=1, keepdims = True)

"""
[results]
y = [[-5, -4, -3], [-11, -10, -9]]
"""
```

In [3]:
from dezero import utils
import dezero.functions as F
from dezero import Function, Variable
import numpy as np

class Sum(Function):
  def __init__(self, axis, keepdims):
    self.axis = axis
    self.keepdims = keepdims
    super().__init__()

  def forward(self, x):
    self.x_shape = x.shape
    y = x.sum(axis=self.axis, keepdims=self.keepdims)
    return y

  def backward(self, gy):
    gy = utils.reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims)
    gx = broadcast_to(gy, self.x_shape)
    return gx

def sum(x, axis=None, keepdims=False):
    return Sum(axis, keepdims)(x)

# test
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = sum(x, axis=0)
print(y)

y.backward()
print(x.grad)

variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])


### Step 40. 브로드캐스트 함수

이번에는 넘파이의 브로드 캐스팅 기능을 Dezero에 도입시키기 위해, `BroadcastTo `, `SumTo` 클래스를 구현해보겠다. 먼저, 넘파이에 `np.braodcast_to()` 함수의 사용법을 알아보겠다.

```python
x = np.array([1, 2, 3])
y = np.broadcast_to(x, (2, 3))
print(y)

"""
[result]
y = np.array([[1, 2, 3], [1, 2, 3]])
"""
```

`BroadcastTo` 클래스는 정확히 `np.broadcast_to()`함수와 동일한 기능을 하며, `SumTo` 클래스는 `BroadcastTo` 클래스의 미분 기능을 하며, 펼려놓은 shape을 다시 압축시킨다.

In [2]:
from dezero import *

# broadcast_to
class BroadcastTo(Function):
  def __init__(self, shape):
    self.shape = shape
    super().__init__()

  def forward(self, x):
    self.x_shape = x.shape
    y = np.broadcast_to(x, self.shape)
    return y

  def backward(self, gy):
    gx = sum_to(gy, self.x_shape)
    return gx

def broadcast_to(x, shape):
  if x.shape == shape:
    return as_variable(x)
  return BroadcastTo(shape)(x)

# sum_to
class SumTo(Function):
  def __init__(self, shape):
    x.shape = shape
    super().__init__()

  def forward(self, x):
    self.x_shape = x.shape
    y = utils.sum_to(x, self.x_shape)
    return y

  def backward(self, gy):
    gx = broadcast_to(gy, self.x_shape)
    return gx

def sum_to(x, shape):
  if x.shape == shape:
    return as_variable(x)
  return SumTo(shape)(x)

이제 Dezero도 넘파이의 브로드캐스팅 기능을 사용할 수 있게 되었다. 제대로 작동하는지 시험해 보도록 하자. 

In [4]:
x0 = Variable(np.array([1, 2, 3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
print(x1.grad)

variable([11 12 13])
variable([3])


### Step 41. 행렬의 곱

이번에는 딥러닝 모델을 구현할 때, 반드시 필요한 행렬 곱 클래스를 구현해보겠다.

In [8]:
class MatMul(Function):
  def forward(self, x, W):
    y = np.dot(x, W)
    return y

  def backward(self, gy):
    x, W = self.inputs
    gx = matmul(gy, W.T)
    gW = matmul(x.T, gy)
    return gx, gW

def matmul(x, W):
  return MatMul()(x, W)
  
x = Variable(np.random.randn(2, 3))
W = Variable(np.random.randn(3, 4))
y = matmul(x, W)
y.backward()

print(x.grad.shape)
print(W.grad.shape)

(2, 3)
(3, 4)
